# 🧪 Class Imbalance Strategies - CTI-BERT ile Adım Adım Test

Her stratejiyi **bağımsız** olarak test et, sonuçları karşılaştır.

## 🤖 Model: CTI-BERT (IBM Research)
- **ibm-research/CTI-BERT**: Cyber Threat Intelligence verisiyle önceden eğitilmiş domain-specific BERT
- **Avantaj**: Güvenlik ve CTI metinlerini anlamada genel BERT'ten daha iyi
- **Reference**: https://huggingface.co/ibm-research/CTI-BERT

## 📊 Dataset: Single Source
- **tumeteor/Security-TTP-Mapping** (14.9k train + 2.6k test)
- **Özellik**: MITRE ATT&CK technique ID'leri (T-codes)
- **Avantaj**: Tutarlı label format, yüksek kalite

## 💡 Kullanım:
1. Setup hücrelerini çalıştır (1-4)
2. İstediğin stratejiyi seç (Strategy 1-5)
3. Sadece o stratejiyi çalıştır (~30-45 dk)
4. Results Comparison hücresini çalıştır
5. Başka bir strateji test et

**Avantaj:** Her stratejiyi ayrı ayrı test edip sonuçları görebilirsin!


## 🔧 SETUP (Her zaman çalıştır)

### 1. Colab Setup

In [ ]:
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Google Colab ortamı tespit edildi")
    
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  GPU bulunamadı! Runtime > Change runtime type > GPU seçin")
    
    print("\n📥 Proje indiriliyor...")
    !rm -rf Mitre_Attack_TTP_Mapping
    !git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git
    os.chdir('Mitre_Attack_TTP_Mapping')
    print(f"✅ Çalışma dizini: {os.getcwd()}")
    
    print("\n📦 Paketler yükleniyor...")
    !pip install -q torch transformers datasets scikit-learn pandas tqdm matplotlib seaborn
    print("✅ Tüm paketler yüklendi")
    
    # HuggingFace bağlantı optimizasyonu
    print("\n🔧 HuggingFace cache ayarları...")
    
    # Create cache directory
    cache_dir = '/content/hf_cache'
    os.makedirs(cache_dir, exist_ok=True)
    
    # Set environment variables
    os.environ['HF_HOME'] = cache_dir
    os.environ['TRANSFORMERS_CACHE'] = cache_dir
    os.environ['HF_DATASETS_CACHE'] = cache_dir
    os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '600'  # 10 minutes
    os.environ['CURL_CA_BUNDLE'] = ''
    os.environ['HF_ENDPOINT'] = 'https://huggingface.co'
    os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'  # Faster downloads
    
    print(f"✅ Cache dizini oluşturuldu: {cache_dir}")
    print(f"   Timeout: 10 dakika")
    
    # Test HuggingFace connection
    try:
        from huggingface_hub import HfApi
        api = HfApi()
        print("\n📡 HuggingFace bağlantı testi...")
        info = api.model_info("ibm-research/CTI-BERT", timeout=30)
        print(f"✅ Model erişilebilir: {info.modelId}")
    except Exception as e:
        print(f"⚠️  Bağlantı uyarısı: {str(e)[:100]}")
        print("   Model indirme denemeye devam edilecek...")
else:
    print("ℹ️  Yerel ortamda çalışıyorsunuz")

### 2. Import Modules

In [ ]:
# Clear import cache
import sys
if 'src.data_loader' in sys.modules:
    del sys.modules['src.data_loader']
if 'src.model' in sys.modules:
    del sys.modules['src.model']
if 'src.train' in sys.modules:
    del sys.modules['src.train']
if 'src.evaluate' in sys.modules:
    del sys.modules['src.evaluate']
if 'src.strategies' in sys.modules:
    del sys.modules['src.strategies']

import torch
import numpy as np
import json
from datetime import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Import prepare_data first
from src.data_loader import prepare_data

# CRITICAL FIX: Define the function HERE if import fails
try:
    from src.data_loader import load_datasets_and_prepare_dataloaders
    print("✅ Fonksiyon GitHub'dan import edildi")
except ImportError:
    print("⚠️  GitHub import başarısız, fonksiyon notebook'ta tanımlanıyor...")
    
    def load_datasets_and_prepare_dataloaders(
        model_name: str = "ibm-research/CTI-BERT",
        batch_size: int = 16,
        max_length: int = 512,
        use_hybrid: bool = True,
        dataset_name: str = "tumeteor/Security-TTP-Mapping"
    ):
        """Wrapper for prepare_data - notebook fallback version."""
        data = prepare_data(
            model_name=model_name,
            max_length=max_length,
            use_hybrid=use_hybrid,
            dataset_name=dataset_name
        )
        
        train_loader = DataLoader(
            data['train_dataset'],
            batch_size=batch_size,
            shuffle=True
        )
        
        return (
            train_loader,
            None,
            data['test_dataset'],
            data['label_list']
        )
    
    print("✅ Fallback fonksiyon tanımlandı (CTI-BERT)")

from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model
from src.strategies import get_strategy_config

print("✅ Modüller yüklendi")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")


---

## 🔧 CONFIGURATION

In [ ]:
# Base training configuration
BASE_CONFIG = {
    'model_name': 'ibm-research/CTI-BERT',  # CTI domain-specific BERT
    'batch_size': 16,
    'learning_rate': 2e-5,
    'num_epochs': 3,
    'max_length': 128,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Output directory
OUTPUT_DIR = Path('outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

# Store results from all strategies
all_test_results = {}

print("✅ Konfigürasyon ayarlandı")
print(f"Model: {BASE_CONFIG['model_name']}")
print(f"Device: {BASE_CONFIG['device']}")
print(f"Output: {OUTPUT_DIR.absolute()}")


---

## 📊 DATA LOADING

In [ ]:
print("📥 Veri yükleniyor...")
print("📦 Dataset: tumeteor/Security-TTP-Mapping (Single Source)")
print(f"🤖 Model: {BASE_CONFIG['model_name']}")
print("")

# Use single dataset: tumeteor only
train_dataloader, val_dataloader, test_dataset, label_names = load_datasets_and_prepare_dataloaders(
    model_name=BASE_CONFIG['model_name'],
    batch_size=BASE_CONFIG['batch_size'],
    max_length=BASE_CONFIG['max_length'],
    use_hybrid=False,  # Single dataset: tumeteor only
    dataset_name="tumeteor/Security-TTP-Mapping"
)

# Get train_dataset from dataloader for strategies
train_dataset = train_dataloader.dataset

# Create test dataloader
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BASE_CONFIG['batch_size'],
    shuffle=False
)

num_labels = len(label_names)

# Create data dict for backward compatibility
data = {
    'train_dataset': train_dataset,
    'test_dataset': test_dataset,
    'label_list': label_names,
    'num_labels': num_labels
}

print(f"✅ Veri yüklendi")
print(f"   Train batches: {len(train_dataloader)}")
print(f"   Test batches: {len(test_dataloader)}")
print(f"   Toplam label sayısı: {num_labels}")
print(f"   İlk 5 label: {label_names[:5]}")


---

## 🧪 STRATEGY TESTING

Her strateji bağımsız olarak test edilebilir. İstediğiniz hücreyi çalıştırın.

### Strategy 1: Baseline (Standard BCE Loss)

**Açıklama:** Standart Binary Cross-Entropy loss kullanır. Referans performans için baseline.

**Önerilen Kullanım:** İlk olarak bu stratejiyi çalıştırın (30-45 dakika).

In [ ]:
strategy_name = "baseline"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 1: Baseline BCE")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['pos_weight'] is not None:
    print(f"   Pos weight: min={strategy_config['pos_weight'].min():.2f}, max={strategy_config['pos_weight'].max():.2f}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 1 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 2: Weighted BCE Loss

**Açıklama:** Her label için frekans bazlı ağırlık hesaplar (pos_weight=458 for rare labels). Class imbalance için en etkili yöntem.

**Önerilen Kullanım:** Baseline'dan sonra bu stratejiyi test edin. F1 > 0.15 ise, diğer stratejileri test etmeye gerek yok.

In [ ]:
strategy_name = "weighted"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 2: Weighted BCE")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['pos_weight'] is not None:
    print(f"   Pos weight: min={strategy_config['pos_weight'].min():.2f}, max={strategy_config['pos_weight'].max():.2f}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 2 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 3: Focal Loss (γ=2, α=0.25)

**Açıklama:** Focal Loss with moderate focusing (γ=2). Hard örneklere odaklanır.

**Önerilen Kullanım:** Weighted BCE başarısız olursa deneyin.

In [ ]:
strategy_name = "focal_weak"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 3: Focal Loss (γ=2)")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['use_focal_loss']:
    print(f"   Focal alpha: {strategy_config.get('focal_alpha')}")
    print(f"   Focal gamma: {strategy_config.get('focal_gamma')}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 3 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 4: Focal Loss (γ=5, α=0.25)

**Açıklama:** Focal Loss with strong focusing (γ=5). Çok hard örneklere odaklanır.

**Önerilen Kullanım:** Focal γ=2 başarılı olursa daha güçlü versiyon için deneyin.

In [ ]:
strategy_name = "focal_strong"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 4: Focal Loss (γ=5)")
print(f"{'='*60}\n")

# Get strategy configuration
strategy_config = get_strategy_config(
    strategy_name=strategy_name,
    train_dataset=data['train_dataset'],
    num_labels=num_labels,
    label_list=label_names
)

# Create DataLoader
if strategy_config['custom_dataloader'] is not None:
    strategy_train_dataloader = strategy_config['custom_dataloader'](BASE_CONFIG['batch_size'])
else:
    strategy_train_dataloader = DataLoader(
        strategy_config['dataset'],
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )

print("📋 Konfigürasyon:")
print(f"   Strategy: {strategy_config['name']}")
print(f"   Description: {strategy_config['description']}")
print(f"   Num labels: {strategy_config['num_labels']}")
print(f"   Focal loss: {strategy_config['use_focal_loss']}")
if strategy_config['use_focal_loss']:
    print(f"   Focal alpha: {strategy_config.get('focal_alpha')}")
    print(f"   Focal gamma: {strategy_config.get('focal_gamma')}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=BASE_CONFIG['model_name'],
    num_labels=strategy_config['num_labels'],
    device=BASE_CONFIG['device'],
    use_focal_loss=strategy_config['use_focal_loss'],
    focal_alpha=strategy_config.get('focal_alpha', 0.25),
    focal_gamma=strategy_config.get('focal_gamma', 2.0),
    pos_weight=strategy_config['pos_weight']
)

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=strategy_train_dataloader,
    num_epochs=BASE_CONFIG['num_epochs'],
    learning_rate=BASE_CONFIG['learning_rate'],
    device=BASE_CONFIG['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=BASE_CONFIG['batch_size'],
    device=BASE_CONFIG['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': strategy_config['name'],
    'description': strategy_config['description'],
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 4 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

# Top-K Label Analysis: Test model capacity with different label subset sizes
print(f"\n{'='*60}")
print(f"🧪 TOP-K LABEL ANALYSIS")
print(f"{'='*60}\n")

from src.strategies import filter_top_k_labels

# Test different K values
k_values = [100, 50, 20, 10, 5]
topk_results = {}

for k in k_values:
    print(f"\n{'='*60}")
    print(f"🔬 Testing Top-{k} Labels")
    print(f"{'='*60}\n")
    
    # Filter dataset to top-k labels
    filtered_train_ds, filtered_label_list, label_mapping = filter_top_k_labels(
        data['train_dataset'], 
        label_names, 
        k=k
    )
    filtered_test_ds, _, _ = filter_top_k_labels(
        data['test_dataset'], 
        label_names, 
        k=k
    )
    
    print(f"📊 Dataset Statistics:")
    print(f"   Top-{k} labels selected")
    print(f"   Train samples: {len(filtered_train_ds)}")
    print(f"   Test samples: {len(filtered_test_ds)}")
    print(f"   Labels: {filtered_label_list[:5]}...")
    
    # Create dataloaders
    topk_train_loader = DataLoader(
        filtered_train_ds,
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )
    topk_test_loader = DataLoader(
        filtered_test_ds,
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=False
    )
    
    # Load model for this K
    print(f"\n🔧 Loading model for {k} labels...")
    topk_model = load_model(
        model_name=BASE_CONFIG['model_name'],
        num_labels=k,
        device=BASE_CONFIG['device'],
        use_focal_loss=False,
        pos_weight=None
    )
    
    # Train model
    print(f"\n🚀 Training on Top-{k}...")
    topk_history = train_model(
        model=topk_model,
        train_dataloader=topk_train_loader,
        num_epochs=BASE_CONFIG['num_epochs'],
        learning_rate=BASE_CONFIG['learning_rate'],
        device=BASE_CONFIG['device']
    )
    
    # Evaluate model
    print(f"\n📊 Evaluating Top-{k}...")
    topk_test_results = evaluate_model(
        model=topk_model,
        test_dataloader=topk_test_loader,
        label_names=filtered_label_list,
        device=BASE_CONFIG['device']
    )
    
    # Store results
    topk_results[f'top_{k}'] = {
        'k': k,
        'num_train': len(filtered_train_ds),
        'num_test': len(filtered_test_ds),
        'metrics': topk_test_results,
        'labels': filtered_label_list
    }
    
    # Display results
    print(f"\n✅ Top-{k} Results:")
    for metric, value in topk_test_results.items():
        if isinstance(value, (int, float)):
            print(f"   {metric}: {value:.4f}")

# Create comparison DataFrame
print(f"\n{'='*60}")
print(f"📊 TOP-K COMPARISON TABLE")
print(f"{'='*60}\n")

topk_comparison = []
for key, data in topk_results.items():
    metrics = data['metrics']
    topk_comparison.append({
        'K': data['k'],
        'Train Samples': data['num_train'],
        'Test Samples': data['num_test'],
        'Micro F1': metrics.get('micro_f1', 0),
        'Hamming Loss': metrics.get('hamming_loss', 0),
        'Micro Precision': metrics.get('micro_precision', 0),
        'Micro Recall': metrics.get('micro_recall', 0),
        'Recall@5': metrics.get('recall_at_5', 0),
        'Precision@5': metrics.get('precision_at_5', 0)
    })

df_topk = pd.DataFrame(topk_comparison)
df_topk = df_topk.sort_values('K', ascending=False)
print(df_topk.to_string(index=False))

# Save CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
topk_csv = OUTPUT_DIR / f'topk_analysis_{timestamp}.csv'
df_topk.to_csv(topk_csv, index=False)
print(f"\n✅ CSV saved: {topk_csv}")

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: F1 Score vs K
ax = axes[0, 0]
ax.plot(df_topk['K'], df_topk['Micro F1'], marker='o', linewidth=2, markersize=8, label='Micro F1', color='blue')
ax.set_xlabel('Number of Labels (K)', fontsize=12)
ax.set_ylabel('Micro F1 Score', fontsize=12)
ax.set_title('Model Performance vs Label Count', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_xaxis()  # Higher K on left

# Plot 2: Hamming Loss vs K
ax = axes[0, 1]
ax.plot(df_topk['K'], df_topk['Hamming Loss'], marker='s', linewidth=2, markersize=8, label='Hamming Loss', color='red')
ax.set_xlabel('Number of Labels (K)', fontsize=12)
ax.set_ylabel('Hamming Loss (lower is better)', fontsize=12)
ax.set_title('Hamming Loss vs Label Count', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_xaxis()  # Higher K on left

# Plot 3: Recall@5 vs K
ax = axes[1, 0]
ax.bar(range(len(df_topk)), df_topk['Recall@5'], alpha=0.7, color='coral')
ax.set_xticks(range(len(df_topk)))
ax.set_xticklabels([f'Top-{k}' for k in df_topk['K']])
ax.set_ylabel('Recall@5', fontsize=12)
ax.set_title('Top-5 Recall by Label Count', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(df_topk['Recall@5']):
    ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10)

# Plot 4: All metrics comparison
ax = axes[1, 1]
x = range(len(df_topk))
width = 0.2
ax.bar([i - width*1.5 for i in x], df_topk['Micro F1'], width, label='F1', alpha=0.8)
ax.bar([i - width*0.5 for i in x], df_topk['Micro Precision'], width, label='Precision', alpha=0.8)
ax.bar([i + width*0.5 for i in x], df_topk['Micro Recall'], width, label='Recall', alpha=0.8)
ax.bar([i + width*1.5 for i in x], df_topk['Recall@5'], width, label='Recall@5', alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels([f'Top-{k}' for k in df_topk['K']])
ax.set_ylabel('Score', fontsize=12)
ax.set_title('All Metrics Comparison', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / f'topk_analysis_{timestamp}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Histogram saved: {OUTPUT_DIR / f'topk_analysis_{timestamp}.png'}")

# Store in all_test_results for comparison
for key, data in topk_results.items():
    all_test_results[key] = {
        'config': f"Top-{data['k']} Labels",
        'description': f"Baseline BCE with {data['k']} most frequent labels",
        'results': data['metrics']
    }

print(f"\n{'='*60}")
print(f"✅ TOP-K ANALYSIS COMPLETE")
print(f"{'='*60}\n")

In [ ]:
# Top-K Label Analysis: Test model capacity with different label subset sizes
print(f"\n{'='*60}")
print(f"🧪 TOP-K LABEL ANALYSIS")
print(f"{'='*60}\n")

from src.strategies import filter_top_k_labels

# Test different K values
k_values = [100, 50, 20, 10, 5]
topk_results = {}

for k in k_values:
    print(f"\n{'='*60}")
    print(f"🔬 Testing Top-{k} Labels")
    print(f"{'='*60}\n")
    
    # Filter dataset to top-k labels
    filtered_train_ds, filtered_label_list, label_mapping = filter_top_k_labels(
        data['train_dataset'], 
        label_names, 
        k=k
    )
    filtered_test_ds, _, _ = filter_top_k_labels(
        data['test_dataset'], 
        label_names, 
        k=k
    )
    
    print(f"📊 Dataset Statistics:")
    print(f"   Top-{k} labels selected")
    print(f"   Train samples: {len(filtered_train_ds)}")
    print(f"   Test samples: {len(filtered_test_ds)}")
    print(f"   Labels: {filtered_label_list[:5]}...")
    
    # Create dataloaders
    topk_train_loader = DataLoader(
        filtered_train_ds,
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=True
    )
    topk_test_loader = DataLoader(
        filtered_test_ds,
        batch_size=BASE_CONFIG['batch_size'],
        shuffle=False
    )
    
    # Load model for this K
    print(f"\n🔧 Loading model for {k} labels...")
    topk_model = load_model(
        model_name=BASE_CONFIG['model_name'],
        num_labels=k,
        device=BASE_CONFIG['device'],
        use_focal_loss=False,
        pos_weight=None
    )
    
    # Train model
    print(f"\n🚀 Training on Top-{k}...")
    topk_history = train_model(
        model=topk_model,
        train_dataloader=topk_train_loader,
        num_epochs=BASE_CONFIG['num_epochs'],
        learning_rate=BASE_CONFIG['learning_rate'],
        device=BASE_CONFIG['device']
    )
    
    # Evaluate model
    print(f"\n📊 Evaluating Top-{k}...")
    topk_test_results = evaluate_model(
        model=topk_model,
        test_dataloader=topk_test_loader,
        label_names=filtered_label_list,
        device=BASE_CONFIG['device']
    )
    
    # Store results
    topk_results[f'top_{k}'] = {
        'k': k,
        'num_train': len(filtered_train_ds),
        'num_test': len(filtered_test_ds),
        'metrics': topk_test_results,
        'labels': filtered_label_list
    }
    
    # Display results
    print(f"\n✅ Top-{k} Results:")
    for metric, value in topk_test_results.items():
        if isinstance(value, (int, float)):
            print(f"   {metric}: {value:.4f}")

# Create comparison DataFrame
print(f"\n{'='*60}")
print(f"📊 TOP-K COMPARISON TABLE")
print(f"{'='*60}\n")

topk_comparison = []
for key, data in topk_results.items():
    metrics = data['metrics']
    topk_comparison.append({
        'K': data['k'],
        'Train Samples': data['num_train'],
        'Test Samples': data['num_test'],
        'Micro F1': metrics.get('micro_f1', 0),
        'Micro Precision': metrics.get('micro_precision', 0),
        'Micro Recall': metrics.get('micro_recall', 0),
        'Recall@5': metrics.get('recall_at_5', 0),
        'Precision@5': metrics.get('precision_at_5', 0)
    })

df_topk = pd.DataFrame(topk_comparison)
df_topk = df_topk.sort_values('K', ascending=False)
print(df_topk.to_string(index=False))

# Save CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
topk_csv = OUTPUT_DIR / f'topk_analysis_{timestamp}.csv'
df_topk.to_csv(topk_csv, index=False)
print(f"\n✅ CSV saved: {topk_csv}")

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: F1 Score vs K
ax = axes[0, 0]
ax.plot(df_topk['K'], df_topk['Micro F1'], marker='o', linewidth=2, markersize=8, label='Micro F1')
ax.set_xlabel('Number of Labels (K)', fontsize=12)
ax.set_ylabel('Micro F1 Score', fontsize=12)
ax.set_title('Model Performance vs Label Count', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_xaxis()  # Higher K on left

# Plot 2: Precision vs Recall
ax = axes[0, 1]
ax.scatter(df_topk['Micro Recall'], df_topk['Micro Precision'], 
           s=200, alpha=0.6, c=df_topk['K'], cmap='viridis')
for i, k in enumerate(df_topk['K']):
    ax.annotate(f'K={k}', 
                (df_topk.iloc[i]['Micro Recall'], df_topk.iloc[i]['Micro Precision']),
                fontsize=10, ha='right')
ax.set_xlabel('Micro Recall', fontsize=12)
ax.set_ylabel('Micro Precision', fontsize=12)
ax.set_title('Precision-Recall Trade-off', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 3: Recall@5 vs K
ax = axes[1, 0]
ax.bar(range(len(df_topk)), df_topk['Recall@5'], alpha=0.7, color='coral')
ax.set_xticks(range(len(df_topk)))
ax.set_xticklabels([f'Top-{k}' for k in df_topk['K']])
ax.set_ylabel('Recall@5', fontsize=12)
ax.set_title('Top-5 Recall by Label Count', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate(df_topk['Recall@5']):
    ax.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10)

# Plot 4: All metrics comparison
ax = axes[1, 1]
x = range(len(df_topk))
width = 0.2
ax.bar([i - width*1.5 for i in x], df_topk['Micro F1'], width, label='F1', alpha=0.8)
ax.bar([i - width*0.5 for i in x], df_topk['Micro Precision'], width, label='Precision', alpha=0.8)
ax.bar([i + width*0.5 for i in x], df_topk['Micro Recall'], width, label='Recall', alpha=0.8)
ax.bar([i + width*1.5 for i in x], df_topk['Recall@5'], width, label='Recall@5', alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels([f'Top-{k}' for k in df_topk['K']])
ax.set_ylabel('Score', fontsize=12)
ax.set_title('All Metrics Comparison', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / f'topk_analysis_{timestamp}.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Histogram saved: {OUTPUT_DIR / f'topk_analysis_{timestamp}.png'}")

# Store in all_test_results for comparison
for key, data in topk_results.items():
    all_test_results[key] = {
        'config': f"Top-{data['k']} Labels",
        'description': f"Baseline BCE with {data['k']} most frequent labels",
        'results': data['metrics']
    }

print(f"\n{'='*60}")
print(f"✅ TOP-K ANALYSIS COMPLETE")
print(f"{'='*60}\n")

---

## 📊 RESULTS COMPARISON

Çalıştırdığınız stratejilerin sonuçlarını karşılaştırmak için bu hücreyi çalıştırın.

In [ ]:
if len(all_test_results) == 0:
    print("⚠️ Henüz hiçbir strateji test edilmedi!")
else:
    print("\n" + "="*80)
    print("📊 STRATEGY COMPARISON RESULTS")
    print("="*80 + "\n")
    
    # Create comparison table
    comparison_data = []
    for strategy_name, data in all_test_results.items():
        results = data['results']
        comparison_data.append({
            'Strategy': strategy_name,
            'Micro F1': results.get('micro_f1', 0),
            'Hamming Loss': results.get('hamming_loss', 0),
            'Recall@5': results.get('recall_at_5', 0),
            'Precision@5': results.get('precision_at_5', 0),
            'Recall@10': results.get('recall_at_10', 0),
            'Precision@10': results.get('precision_at_10', 0),
            'Example Accuracy': results.get('example_based_accuracy', 0)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    df_comparison = df_comparison.sort_values('Micro F1', ascending=False)
    
    print(df_comparison.to_string(index=False))
    print("\n" + "="*80)
    
    # Find best strategy
    best_strategy = df_comparison.iloc[0]['Strategy']
    best_f1 = df_comparison.iloc[0]['Micro F1']
    best_hamming = df_comparison.iloc[0]['Hamming Loss']
    
    print(f"\n🏆 EN İYİ STRATEJİ: {best_strategy}")
    print(f"   Micro F1 Score: {best_f1:.4f}")
    print(f"   Hamming Loss: {best_hamming:.4f} (lower is better)")
    
    # Compare to baseline if exists
    if 'baseline' in all_test_results:
        baseline_f1 = all_test_results['baseline']['results'].get('micro_f1', 0)
        improvement = ((best_f1 - baseline_f1) / baseline_f1 * 100) if baseline_f1 > 0 else 0
        print(f"   Baseline'a göre iyileştirme: {improvement:+.2f}%")
    
    # Plot comparison
    if len(all_test_results) > 1:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # F1 scores comparison
        strategies = df_comparison['Strategy'].tolist()
        micro_f1 = df_comparison['Micro F1'].tolist()
        recall_at_5 = df_comparison['Recall@5'].tolist()
        
        x = np.arange(len(strategies))
        width = 0.35
        
        axes[0].bar(x - width/2, micro_f1, width, label='Micro F1', alpha=0.8)
        axes[0].bar(x + width/2, recall_at_5, width, label='Recall@5', alpha=0.8)
        axes[0].set_xlabel('Strategy')
        axes[0].set_ylabel('Score')
        axes[0].set_title('Micro F1 & Recall@5 Comparison')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(strategies, rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(axis='y', alpha=0.3)
        
        # Hamming Loss comparison
        hamming_loss_vals = df_comparison['Hamming Loss'].tolist()
        
        axes[1].bar(x, hamming_loss_vals, alpha=0.8, color='coral')
        axes[1].set_xlabel('Strategy')
        axes[1].set_ylabel('Hamming Loss (lower is better)')
        axes[1].set_title('Hamming Loss Comparison')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(strategies, rotation=45, ha='right')
        axes[1].grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\n")

if len(all_test_results) == 0:
    print("⚠️ Kaydedilecek sonuç yok!")
else:
    # Helper function to convert numpy arrays to lists
    def convert_to_serializable(obj):
        """Recursively convert numpy arrays to lists for JSON serialization."""
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_to_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, (np.integer, np.floating)):
            return float(obj)
        else:
            return obj
    
    # Save results to JSON
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.json'
    
    # Convert results to JSON-serializable format
    serializable_results = convert_to_serializable(all_test_results)
    
    with open(results_file, 'w') as f:
        json.dump(serializable_results, f, indent=2)
    
    print(f"✅ Sonuçlar kaydedildi: {results_file}")
    
    # Save comparison table to CSV
    comparison_data = []
    for strategy_name, data in all_test_results.items():
        results = data['results']
        comparison_data.append({
            'Strategy': strategy_name,
            'Micro_F1': results.get('micro_f1', 0),
            'Hamming_Loss': results.get('hamming_loss', 0),
            'Recall_at_5': results.get('recall_at_5', 0),
            'Precision_at_5': results.get('precision_at_5', 0),
            'Recall_at_10': results.get('recall_at_10', 0),
            'Precision_at_10': results.get('precision_at_10', 0),
            'Example_Based_Accuracy': results.get('example_based_accuracy', 0)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    csv_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.csv'
    df_comparison.to_csv(csv_file, index=False)
    
    print(f"✅ CSV tablosu kaydedildi: {csv_file}")
    
    # Download files if on Colab
    try:
        from google.colab import files
        files.download(str(results_file))
        files.download(str(csv_file))
        print("✅ Dosyalar indirildi")
    except ImportError:
        print("💡 Colab değil, dosyalar sadece kaydedildi")

In [ ]:
if len(all_test_results) == 0:
    print("⚠️ Kaydedilecek sonuç yok!")
else:
    # Helper function to convert numpy arrays to lists
    def convert_to_serializable(obj):
        """Recursively convert numpy arrays to lists for JSON serialization."""
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_to_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, (np.integer, np.floating)):
            return float(obj)
        else:
            return obj
    
    # Save results to JSON
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.json'
    
    # Convert results to JSON-serializable format
    serializable_results = convert_to_serializable(all_test_results)
    
    with open(results_file, 'w') as f:
        json.dump(serializable_results, f, indent=2)
    
    print(f"✅ Sonuçlar kaydedildi: {results_file}")
    
    # Save comparison table to CSV
    comparison_data = []
    for strategy_name, data in all_test_results.items():
        results = data['results']
        comparison_data.append({
            'Strategy': strategy_name,
            'Micro_F1': results.get('micro_f1', 0),
            'Recall_at_5': results.get('recall_at_5', 0),
            'Precision_at_5': results.get('precision_at_5', 0),
            'Recall_at_10': results.get('recall_at_10', 0),
            'Precision_at_10': results.get('precision_at_10', 0),
            'Example_Based_Accuracy': results.get('example_based_accuracy', 0)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    csv_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.csv'
    df_comparison.to_csv(csv_file, index=False)
    
    print(f"✅ CSV tablosu kaydedildi: {csv_file}")
    
    # Download files if on Colab
    try:
        from google.colab import files
        files.download(str(results_file))
        files.download(str(csv_file))
        print("✅ Dosyalar indirildi")
    except ImportError:
        print("💡 Colab değil, dosyalar sadece kaydedildi")

---

## 📝 NOTLAR VE ÖNERİLER

**Model:**
- **CTI-BERT** (ibm-research/CTI-BERT): Cyber Threat Intelligence'a özel BERT
- Güvenlik metinlerinde genel BERT'ten daha iyi performans
- Pre-trained on security-specific corpus

**Dataset:**
- Hybrid mode kullanılıyor (tumeteor + sarahwei + Zainabsa99)
- Toplam ~16k sample, daha iyi label coverage
- Tek dataset kullanmak için: `use_hybrid=False` parametresi ekle

**Test Sırası:**
1. **Baseline** - Her zaman ilk olarak çalıştırın (referans performans)
2. **Weighted BCE** - En umut verici strateji, Micro F1 > 0.15 bekleniyor
3. Weighted BCE başarılı olursa → Bitti, kullan!
4. Weighted BCE başarısız olursa → **Focal Loss (γ=2)** dene
5. Hala kötüyse → **Focal Loss (γ=5)** dene
6. Hepsi başarısız olursa → **Top-100** ile model öğrenme kapasitesini kontrol et

**Başarı Kriterleri:**
- Micro F1 > 0.15 → İyi performans
- Recall@5 > 0.30 → Çok iyi performans
- Micro F1 < 0.10 → Strateji işe yaramadı

**CTI-BERT ile Beklenen İyileşmeler:**
- Cyber security terminolojisini daha iyi anlama
- MITRE ATT&CK kavramlarında daha yüksek accuracy
- Rare technique'leri daha iyi tanıma

**Metrikler:**
- **Micro F1:** Genel performans (threshold-based)
- **Recall@5:** Top-5 tahmin içinde kaç doğru label var
- **Precision@5:** Top-5 tahminlerin kaçı doğru
- **Example-Based Accuracy:** Tüm label'ların tam eşleşmesi (çok strict)

**Süre:**
- Her strateji ~30-45 dakika sürer
- CTI-BERT model download ilk seferde ~5 dakika ekstra
- Modular yaklaşım sayesinde istediğiniz zaman durup sonuçlara bakabilirsiniz

**Hatırlatmalar:**
- Herbir strateji hücresi bağımsızdır, istediğiniz sırada çalıştırabilirsiniz
- Sonuçlar `all_test_results` dictionary'sinde saklanır
- Comparison hücresini dilediğiniz zaman çalıştırıp ara sonuçlara bakabilirsiniz
- CTI-BERT ilk indirilirken cache'lenir, sonraki çalıştırmalar hızlıdır
